In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
from torchvision import transforms
from torchvision.datasets.folder import default_loader, has_file_allowed_extension
from torch.utils.data import Dataset
import torch.utils.data as torch_data


class CifarDataLoader:
    """
    For loading Validation data from the ImageNet dataset.
    """

    def __init__(self, image_size: int=32, batch_size: int = 128,
                 is_training: bool = False, num_workers: int = 2, num_samples_per_class: int = None):

        self.batch_size = batch_size

        transform = transforms.Compose([
            # transforms.RandomHorizontalFlip(),
            # transforms.RandomCrop(32, padding=4),
            transforms.Resize((32,32)),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616))
        ])
        if is_training:
            trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
            self._data_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
        else:
            testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
            self._data_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    @property
    def data_loader(self) -> torch_data.DataLoader:
        """
        Returns the data-loader
        """
        return self._data_loader
    
    def __len__(self):
        return len(self.data_loader)

    def __iter__(self):
        return iter(self.data_loader)

In [ ]:
import logging

from tqdm import tqdm
import torch
from torch import nn

# from Examples.torch.utils.image_net_data_loader import ImageNetDataLoader

logger = logging.getLogger('Eval')


class CifarEvaluator:

    def __init__(self):
        self._val_data_loader = CifarDataLoader(is_training=False).data_loader

    def evaluate(self, model: nn.Module, iterations: int = None, use_cuda: bool = False) -> float:
        """
        Evaluate the specified model using the specified number of samples batches from the
        validation set.
        :param model: The model to be evaluated.
        :param iterations: The number of batches to use from the validation set.
        :param use_cuda: If True then use a GPU for inference.
        :return: The accuracy for the sample with the maximum accuracy.
        """

        device = torch.device('cpu')
        if use_cuda:
            if torch.cuda.is_available():
                "here"
                device = torch.device('cuda')
            else:
                logger.error('use_cuda is selected but no cuda device found.')
                raise RuntimeError("Found no CUDA Device while use_cuda is selected")

        if iterations is None:
            logger.info('No value of iteration is provided, running evaluation on complete dataset.')
            iterations = len(self._val_data_loader)
        if iterations <= 0:
            logger.error('Cannot evaluate on %d iterations', iterations)

        correct = 0
        total = 0
        logger.info("Evaluating nn.Module for %d iterations with batch_size %d",
                    iterations, self._val_data_loader.batch_size)

        model = model.to(device)
        model = model.eval()

        with torch.no_grad():
            for i, (input_data, target_data) in tqdm(enumerate(self._val_data_loader), total=iterations):
                if i == iterations:
                    break

                inputs_batch = input_data.to(device)
                target_batch = target_data.to(device)
                predicted_batch = model(inputs_batch)
                _, predicted = predicted_batch.max(1)
                total += target_batch.size(0)
                correct += predicted.eq(target_batch).sum().item()

        return 100 * correct / total


In [6]:
import logging

from tqdm import tqdm
import torch
from torch import nn, optim

logger = logging.getLogger('Trainer')


class CifarTrainer:
    """
    For training a model using the ImageNet dataset.
    """

    # pylint: disable=too-many-arguments
    def __init__(self):

        self._train_loader = CifarDataLoader(is_training=True).data_loader

        self._evaluator = CifarEvaluator()

    def _train_loop(self, model: nn.Module, criterion: torch.nn.modules.loss, optimizer: torch.optim,
                    max_iterations: int, current_epoch: int, max_epochs: int,
                    debug_steps: int = 1000, use_cuda: bool = False):
        """
        Train the specified model using the ImageNet dataset for one epoch.
        :param model: The model to train.
        :param criterion: loss function to optimize
        :param optimizer: optimizer function
        :param max_iterations: total number of iterations to train
        :param current_epoch: current epoch for model training, used for reporting logs in debug steps
        :param max_epochs: total epoch for model training, used for reporting logs in debug steps
        :param debug_steps: number of training iterations to report accuracy/loss metrics
        :param use_cuda: If True then use GPU device

        :return: None
        """
        # pylint: disable-msg=too-many-locals
        device = torch.device('cpu')
        if use_cuda:
            if torch.cuda.is_available():
                device = torch.device('cuda')
            else:
                logger.error('use_cuda is selected but no cuda device found.')
                raise RuntimeError("Found no CUDA Device while use_cuda is selected")

        # switch to training mode
        model = model.to(device)
        model.train()

        avg_loss = 0.0

        for i, (images, target) in tqdm(enumerate(self._train_loader), total=max_iterations):
            if i == max_iterations:
                break

            images = images.to(device)
            target = target.to(device)

            # compute model output
            output = model(images)
            loss = criterion(output, target)
            avg_loss += loss.item()

            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % debug_steps == 0:
                eval_accuracy = self._evaluator.evaluate(model, use_cuda=use_cuda)
                logger.info('Epoch #%d/%d: iteration #%d/%d: Global Avg Loss=%f, Eval Accuracy=%f',
                            current_epoch, max_epochs, i, max_iterations,
                            avg_loss / i, eval_accuracy)
                # switch to training mode after evaluation
                model.train()

        eval_accuracy = self._evaluator.evaluate(model, use_cuda=use_cuda)
        print("eval : ", eval_accuracy)
        logger.info('At the end of Epoch #%d/%d: Global Avg Loss=%f, Eval Accuracy=%f',
                    current_epoch, max_epochs, avg_loss / max_iterations, eval_accuracy)

    def train(self, model: nn.Module, max_epochs: int = 20, learning_rate: int = 0.1,
              weight_decay: float = 1e-4, decay_rate: float = 0.1, learning_rate_schedule: list = None,
              debug_steps: int = 1000, use_cuda: bool = False):
        """
        Train the specified model using the ImageNet dataset.
        :param model: The model to train.
        :param max_epochs: Maximum number of epochs to use in training.
        :param learning_rate: Learning rate
        :param weight_decay: Weight decay
        :param decay_rate: Multiplicative factor of learning rate decay
        :param learning_rate_schedule: Adjust the learning rate based on the number of epochs
        :param debug_steps: number of training iterations to report accuracy/loss metrics
        :param use_cuda: If True then use GPU device

        :return: None
        """
        max_iterations = len(self._train_loader)

        loss = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

        if learning_rate_schedule:
            learning_rate_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, learning_rate_schedule,
                                                                     gamma=decay_rate)

        for current_epoch in range(max_epochs):
            self._train_loop(model, loss, optimizer, max_iterations, current_epoch + 1, max_epochs, debug_steps,
                             use_cuda)

            if learning_rate_schedule:
                learning_rate_scheduler.step()


In [7]:
import os
import torch
from typing import List

class CifarDataPipeline:

    @staticmethod
    def get_val_dataloader() -> torch.utils.data.DataLoader:

        data_loader = CifarDataLoader()
        return data_loader

    @staticmethod
    def evaluate(model: torch.nn.Module, iterations: int, use_cuda: bool) -> float:
        """
        Given a torch model, evaluates its Top-1 accuracy on the dataset
        :param model: the model to evaluate
        :param iterations: the number of batches to be used to evaluate the model. A value of 'None' means the model will be
                           evaluated on the entire dataset once.
        :param use_cuda: whether or not the GPU should be used.
        """
        evaluator = CifarEvaluator()

        return evaluator.evaluate(model, iterations=iterations, use_cuda=use_cuda)

    @staticmethod
    def finetune(model: torch.nn.Module, epochs: int, learning_rate: float, learning_rate_schedule: List, use_cuda: bool):
        """
        Given a torch model, finetunes the model to improve its accuracy
        :param model: the model to finetune
        :param epochs: The number of epochs used during the finetuning step.
        :param learning_rate: The learning rate used during the finetuning step.
        :param learning_rate_schedule: The learning rate schedule used during the finetuning step.
        :param use_cuda: whether or not the GPU should be used.
        """
        trainer = CifarTrainer()

        trainer.train(model, max_epochs=epochs, learning_rate=learning_rate,
                      learning_rate_schedule=learning_rate_schedule, use_cuda=use_cuda)

In [8]:
from resnet import resnet18
model = resnet18(pretrained=True)

In [20]:
accuracy = CifarDataPipeline.evaluate(model, iterations=None, use_cuda=True)
print(accuracy)

Files already downloaded and verified
2024-12-26 01:07:21,063 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-26 01:07:21,064 - Eval - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 102.65it/s]

0.006194380265247973
93.07


In [21]:
from decimal import Decimal
from aimet_torch.defs import GreedySelectionParameters, ChannelPruningParameters
from aimet_common.defs import CompressionScheme, CostMetric

greedy_params = GreedySelectionParameters(target_comp_ratio=Decimal(0.7),
                                          num_comp_ratio_candidates=3)
modules_to_ignore = [model.conv1]
auto_params = ChannelPruningParameters.AutoModeParams(greedy_select_params=greedy_params,
                                                      modules_to_ignore=modules_to_ignore)
data_loader = CifarDataPipeline.get_val_dataloader()
params = ChannelPruningParameters(data_loader=data_loader,
                                  num_reconstruction_samples=10,
                                  allow_custom_downsample_ops=False,
                                  mode=ChannelPruningParameters.Mode.auto,
                                  params=auto_params)

eval_callback = CifarDataPipeline.evaluate
eval_iterations = 1
compress_scheme = CompressionScheme.channel_pruning
cost_metric = CostMetric.mac

Files already downloaded and verified


In [22]:
from aimet_torch.compress import ModelCompressor
compressed_model, comp_stats = ModelCompressor.compress_model(model=model,
                                                              eval_callback=eval_callback,
                                                              eval_iterations=eval_iterations,
                                                              input_shape=(1, 3, 32, 32),
                                                              compress_scheme=compress_scheme,
                                                              cost_metric=cost_metric,
                                                              parameters=params)

print(comp_stats)

2024-12-26 01:08:05,198 - CompRatioSelect - INFO - Analyzing compression ratio: 0.3333333333333333333333333333 =====================>


/usr/local/lib/python3.10/dist-packages/aimet_torch/winnow/mask_propagation_winnower.py:95: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dummy_input = torch.tensor(dummy_input).cuda()  # pylint: disable=not-callable


2024-12-26 01:08:05,679 - ChannelPruning - INFO - finished linear regression fit 
Files already downloaded and verified
2024-12-26 01:08:06,194 - Eval - INFO - Evaluating nn.Module for 1 iterations with batch_size 128


100%|██████████| 1/1 [00:00<00:00, 26.67it/s]

0.008884429931640625
2024-12-26 01:08:06,375 - CompRatioSelect - INFO - Layer layer1.0.conv1, comp_ratio 0.333333 ==> eval_score=92.187500
2024-12-26 01:08:06,375 - CompRatioSelect - INFO - Analyzing compression ratio: 0.6666666666666666666666666666 =====================>


...

2024-12-26 01:09:38,179 - ChannelPruning - INFO - finished linear regression fit 
Files already downloaded and verified
2024-12-26 01:09:38,668 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-26 01:09:38,668 - Eval - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 97.78it/s] 


0.0062369606162928325
Files already downloaded and verified
2024-12-26 01:09:40,098 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-26 01:09:40,099 - Eval - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 94.57it/s] 


0.0058783519117138056
**********************************************************************************************
Compressed Model Statistics
Baseline model accuracy: 93.070000, Compressed model accuracy: 34.130000
Compression ratio for memory=0.364686, mac=0.459084

**********************************************************************************************

Per-layer Stats
    Name:layer1.0.conv1, compression-ratio: 0.3333333333333333333333333333
    Name:layer1.0.conv2, compression-ratio: 0.3333333333333333333333333333
    Name:layer1.1.conv1, compression-ratio: 0.3333333333333333333333333333
    Name:layer1.1.conv2, compression-ratio: 0.3333333333333333333333333333
    Name:layer2.0.conv1, compression-ratio: 0.3333333333333333333333333333
    Name:layer2.0.conv2, compression-ratio: None
    Name:layer2.0.downsample.0, compression-ratio: 0.3333333333333333333333333333
    Name:layer2.1.conv1, compression-ratio: 0.3333333333333333333333333333
    Name:layer2.1.conv2, compression

In [23]:
accuracy = CifarDataPipeline.evaluate(compressed_model, iterations=None, use_cuda=True)
print(accuracy)

Files already downloaded and verified
2024-12-26 01:09:41,567 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-26 01:09:41,568 - Eval - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 96.65it/s] 

0.006051205381562438
34.13


In [24]:

CifarDataPipeline.finetune(compressed_model, epochs=10, learning_rate=10e-4, learning_rate_schedule=[5, 10],
                              use_cuda=True)

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 391/391 [00:07<00:00, 55.36it/s]

2024-12-26 01:09:50,862 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-26 01:09:50,863 - Eval - INFO - Evaluating nn.Module for 79 iterations with batch_size 128



100%|██████████| 79/79 [00:00<00:00, 97.55it/s] 

0.005133218403103985
eval :  77.72
2024-12-26 01:09:51,822 - Eval - INFO - At the end of Epoch #1/10: Global Avg Loss=0.862478, Eval Accuracy=77.720000


...

0.0050340785255915
eval :  87.7
2024-12-26 01:10:57,922 - Eval - INFO - At the end of Epoch #9/10: Global Avg Loss=0.038059, Eval Accuracy=87.700000



100%|██████████| 391/391 [00:07<00:00, 54.15it/s]

2024-12-26 01:11:05,280 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-26 01:11:05,281 - Eval - INFO - Evaluating nn.Module for 79 iterations with batch_size 128



100%|██████████| 79/79 [00:00<00:00, 97.55it/s] 

0.005044475386414347
eval :  87.63
2024-12-26 01:11:06,231 - Eval - INFO - At the end of Epoch #10/10: Global Avg Loss=0.032650, Eval Accuracy=87.630000


In [25]:
accuracy = CifarDataPipeline.evaluate(compressed_model, iterations=None, use_cuda=True)
print(accuracy)

Files already downloaded and verified
2024-12-26 01:11:06,709 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-26 01:11:06,709 - Eval - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 97.88it/s] 

0.005935035174406028
87.63


In [18]:
os.makedirs('./output/', exist_ok=True)
torch.save(compressed_model, './output/finetuned_model')